Here K and tau_i are being calculated using genetic algorithm:

In [ ]:
pip install control
pip install geneticalgorithm

Define the given second order transfer function

In [ ]:
import control as ctrl

num = [1]
den = [20, 10, 1]

# Defining the second order plant transfer function
plant_tf = ctrl.TransferFunction(num, den)

print("Plant Transfer Function:")
print(plant_tf)

Now we define a time vector from 0 to 30 :

In [ ]:
import numpy as np

# Define the time vector
t = np.arange(0, 30.01, 0.01)

print("Time vector:", t)

Now we create an unit step input:

In [ ]:
r = np.ones_like(t)
print("Step input array ", r)

Definitions of the three errors: ITAE, IAE, and ISE:

In [ ]:
def compute_ITAE(K, plant_tf, r, t):
    # Simulate the closed-loop system
    controller_tf = ctrl.TransferFunction([K[0]*K[1], K[0]], [K[1], 0])
    closed_loop_tf = ctrl.feedback(ctrl.series(controller_tf, plant_tf), 1)
    t_sim, y = ctrl.step_response(closed_loop_tf, T=t)

    # Compute ITAE criteria
    error = r - y
    ITAE_value = np.trapz(t_sim * np.abs(error), t_sim)
    return ITAE_value

def compute_IAE(K,plant_tf,r,t):
    # Simulate the closed-loop system
    controller_tf = ctrl.TransferFunction([K[0]*K[1], K[0]], [K[1], 0])
    closed_loop_tf = ctrl.feedback(ctrl.series(controller_tf, plant_tf), 1)
    t_sim, y = ctrl.step_response(closed_loop_tf, T=t)

    # Compute IAE criteria
    error = r - y
    IAE_value = np.trapz(np.abs(error), t_sim)
    return IAE_value

def compute_ISE(K,plant_tf,r,t):
    # Simulate the closed-loop system
    controller_tf = ctrl.TransferFunction([K[0]*K[1], K[0]], [K[1], 0])
    closed_loop_tf = ctrl.feedback(ctrl.series(controller_tf, plant_tf), 1)
    t_sim, y = ctrl.step_response(closed_loop_tf, T=t)

    # Compute ISE criteria
    error = r - y
    ITAE_value = np.trapz(np.abs(error)**2, t_sim)
    return ITAE_value

In [ ]:
from geneticalgorithm import geneticalgorithm as ga

def fitness_function(K):
    objective = compute_ITAE(K,plant_tf, r, t)
    return objective


varbound = np.array([[0, 20.09],[0, 5],[0,5]])

#defining the parameters required for Genetic Algorithm in the following dictionary:
algorithm_param = {'max_num_iteration': 50,\
                   'population_size':10,\
                   'mutation_probability':0.3,\
                   'elit_ratio': 0.01,\
                   'crossover_probability': 1,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':None}

# Initialize the genetic algorithm optimizer
optimizer = ga(function=fitness_function, dimension=3, variable_type='real', variable_boundaries=varbound, algorithm_parameters=algorithm_param)

# Running the genetic algorithm
optimizer.run()

# Get the best solution
best_solution = optimizer.output_dict
best_Params = best_solution['variable'] 

Kp = best_Params[0]
Ki = best_Params[1]
Kd = best_Params[2]

best_objective = best_solution['function'] #minimum ITAE value